
----------------------Task 1----------------------

Build a parser that can extract building/business name from a given address.

building/business name might appears in various format within an address.

Example input address and respective expected building/business name are given in 'sample_building_name_extracted.csv' file

Sample Road Name List also provided in 'names_sample.csv'

Based on that road list you can train custom road model to extract building/business name given in 'user_raw_data.csv'

-------------------------Task 2-----------------------------

Build a parser that can extract building/business name from a given address

building/business name might appears in various format within an address.

Example input address and respective expected road no/road name are given in 'sample_road_name_extracted.csv' file

Sample Road Name List also provided in 'roads.csv'

Based on that road list you can train custom road model to extract road no/road name given in 'user_raw_data.csv

In [0]:
# Install extra-dependencies

#for downloading extended dependencis
import nltk
nltk.download('punkt')
! pip -q install git+https://www.github.com/keras-team/keras-contrib.git sklearn-crfsuite
! pip install keras==2.2.4

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 757kB 4.4MB/s 
     |████████████████████████████████| 317kB 2.7MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
#importing all the necessary liaberies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from nltk.tokenize import WordPunctTokenizer, word_tokenize

import tensorflow as tf
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from sklearn.model_selection import train_test_split
from sklearn_crfsuite.metrics import flat_classification_report

Using TensorFlow backend.


In [0]:
import os
# mounting google drive
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Tasks/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Load all the data for TASK 2:
t2_user_name = pd.read_csv('Task2/roads.csv')
t2_road = pd.read_csv('Task2/sample_road_name_extracted.csv')
t2_raw_user = pd.read_excel('Task2/user_raw_data.xlsx')

In [0]:
"""
Function to pre process the whole dataset. 

Taking input the dataframe and returning a list
"""
def preProcessData(user_data):
  #replacing all \n to space and other uncessary info from scenence and converted to them in lowercase letter
  word_list = []
  user_data = user_data.replace({'\n'},'', regex=True)
  user_data = user_data.loc[:, "address"].str.lower()
  
  for i in range(len(user_data)):
    tokenized_data = nltk.word_tokenize(user_data.iloc[i])
    word_list.append(tokenized_data)
  
  return word_list

In [0]:
#receiving data from preProcessData function
processed_data = preProcessData(t2_raw_user)

word_list = []
sen_list = []
counter = 0
#loping through all the words for converting it into a list
for i in processed_data:
  counter = counter + 1
  sentence = 'sentence-' + str(counter)
  for r in i:
    word_list.append(r)
    sen_list.append(sentence)

dic = {'sentence':sen_list, 'word':word_list}
data = pd.DataFrame(dic)

data.to_excel("pre_processed_output_data.xlsx", index = False)

## Hyperprameters

In [0]:
BATCH_SIZE = 256  # Number of examples used in each iteration
EPOCHS = 25  # Number of passes through entire dataset
MAX_LEN = 15  # Max length of review (in words)
EMBEDDING = 40  # Dimension of word embedding vector

In [0]:
#Data preview
data = pd.read_excel("test_file.xlsx")
print("Number of sentences: ", len(data.groupby(['Sentence #'])))

words = list(set(data["Word"].values))
n_words = len(words)
print("Number of words in the dataset: ", n_words)

tags = list(set(data["Tag"].values))
print("Tags:", tags)
n_tags = len(tags)
print("Number of Labels: ", n_tags)


Number of sentences:  1450
Number of words in the dataset:  3924
Tags: ['B-BLD', 'O', 'I-RAD', 'B-RAD', 'I-BLD']
Number of Labels:  5


In [0]:
class SentenceGetter(object):
    """Class to Get the sentence in this format:"""

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        healper_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(), s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(healper_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        """Return one sentence"""
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
getter = SentenceGetter(data)
sent = getter.get_next()

# Get all the sentences
sentences = getter.sentences

In [0]:
"""
using the `word2idx` dictionary to convert each word to a corresponding integer ID 
"""
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
idx2word = {i: w for w, i in word2idx.items()}

In [0]:
"""
Using the `tag2idx` to convert each tags to a corresponding integer ID
"""
tag2idx = {t: i+1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}


In [87]:
"""
To feed the text into our Bi-LSTM-CRF, all texts should be the same length. 
used the sequence.pad_sequences() method and MAX_LEN variable.
All texts longer than MAX_LEN are truncated and shorter texts are padded to get them to the same length
"""
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0]] for w in s] for s in sentences]
# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])

# Convert Tag/Label to tag_index
y = [[tag2idx[w[1]] for w in s] for s in sentences]
# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])

# One-Hot encode
y = [to_categorical(i, num_classes=n_tags+1) for i in y]  # n_tags+1(PAD)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
X_tr.shape, X_te.shape, np.array(y_tr).shape, np.array(y_te).shape


((1305, 15), (145, 15), (1305, 15, 6), (145, 15, 6))

In [0]:
# Model definition
input = Input(shape=(MAX_LEN,))
model = Embedding(input_dim=n_words+2, output_dim=EMBEDDING, input_length=MAX_LEN, mask_zero=True)(input)  # default: 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)
crf = CRF(n_tags+1)  # CRF layer, n_tags+1(PAD)
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 40)            157040    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 15, 100)           36400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 15, 6)             354       
Total params: 198,844
Trainable params: 198,844
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = model.fit(X_tr, np.array(y_tr), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1, verbose=2)

#to save the model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_road_bld.h5")




Train on 1174 samples, validate on 131 samples
Epoch 1/25





 - 4s - loss: 3.5830 - crf_viterbi_accuracy: 0.0154 - val_loss: 3.5007 - val_crf_viterbi_accuracy: 0.0392
Epoch 2/25
 - 1s - loss: 3.2446 - crf_viterbi_accuracy: 0.3264 - val_loss: 2.9692 - val_crf_viterbi_accuracy: 0.8233
Epoch 3/25
 - 1s - loss: 2.6937 - crf_viterbi_accuracy: 0.8758 - val_loss: 2.6768 - val_crf_viterbi_accuracy: 0.8767
Epoch 4/25
 - 1s - loss: 2.5191 - crf_viterbi_accuracy: 0.8919 - val_loss: 2.6112 - val_crf_viterbi_accuracy: 0.8792
Epoch 5/25
 - 1s - loss: 2.4467 - crf_viterbi_accuracy: 0.8922 - val_loss: 2.5645 - val_crf_viterbi_accuracy: 0.8792
Epoch 6/25
 - 1s - loss: 2.3911 - crf_viterbi_accuracy: 0.8921 - val_loss: 2.5233 - val_crf_viterbi_accuracy: 0.8792
Epoch 7/25
 - 1s - loss: 2.3451 - crf_viterbi_accuracy: 0.8921 - val_loss: 2.4885 - val_crf_viterbi_accuracy: 0.8792
Epoch 8/25
 - 1s - loss: 2.3030 - crf_viterbi_accuracy: 0.8921 - val_loss: 2.4611 - val_crf_viterbi_accuracy: 0.8792
Epoch 9/2

In [0]:
# Eval
pred_cat = model.predict(X_te)
pred = np.argmax(pred_cat, axis=-1)
y_te_true = np.argmax(y_te, -1)

In [0]:
# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in pred]
y_te_true_tag = [[idx2tag[i] for i in row] for row in y_te_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_te_true_tag)
print(report)

              precision    recall  f1-score   support

       B-BLD       0.00      0.00      0.00        19
       B-RAD       0.08      0.05      0.06        20
       I-BLD       0.20      0.03      0.06        31
       I-RAD       0.31      0.29      0.30        34
           O       0.94      0.98      0.96      1133
         PAD       1.00      1.00      1.00       938

    accuracy                           0.95      2175
   macro avg       0.42      0.39      0.40      2175
weighted avg       0.93      0.95      0.94      2175



In [0]:
#formatting the output 
def getPredSentValue(passedVal):
  tempPredList = []
  pre_list = []
  for i in range(15):
    for j in range(6):
      tempPredList.append(int(passedVal[i][j]))
    tempPredList = np.asarray(tempPredList)
  
    product = emp.dot(tempPredList)
    pre_val = inv_tag.get(product)
    pre_list.append(pre_val)
    tempPredList = []
  res = dict(zip(s, pre_list)) 
  return res

In [0]:
#to get the predicted sentence
def getSentList(works_list):
  sent_by_words = []
  for i in range(15):
    temp_l_word = works_list[i]
    sent_by_words.append(inv_map.get(temp_l_word))
  return sent_by_words

In [0]:
#to convert tags in numpy array
emp = []
for ta in inv_tag.keys():
  emp.append(ta)
emp.remove(0)
emp.insert(0,0)
emp = np.asarray(emp)
emp

array([0, 1, 2, 3, 4, 5])

In [78]:
n = 0
temp = model.predict(X_te)
passed_val = temp[n].tolist()
s=getSentList(X_te[n])
res = getPredSentValue(passed_val)
print(res)

{'f-a603': 'O', '45': 'O', 'new': 'O', 'eskaton': 'O', 'beside': 'O', 'biam': 'I-RAD', 'off': 'B-BLD', 'PAD': 'PAD'}


In [80]:
n = 5
temp = model.predict(X_te)
passed_val = temp[n].tolist()
s=getSentList(X_te[n])
res = getPredSentValue(passed_val)
print(res)

{'34': 'O', 'arambag': 'O', 'motijhil': 'O', 'PAD': 'PAD'}


In [91]:
n = 137
temp = model.predict(X_te)
passed_val = temp[n].tolist()
s=getSentList(X_te[n])
res = getPredSentValue(passed_val)
print(res)

{'house-26/15': 'O', ',': 'O', 'haque': 'O', 'villa': 'I-RAD', 'flat-4/c': 'O', 'middle': 'O', 'paikpara': 'O', 'mirpur-1': 'O', 'PAD': 'PAD'}


In [0]:
# inverting word2idx and tag2idx values
inv_map = {v: k for k, v in word2idx.items()}
inv_tag = {v: k for k, v in tag2idx.items()}